In [36]:
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy 
from sqlalchemy import create_engine

#from config import username, password

In [37]:
movie = pd.read_csv('predictions_boxoffice.csv', index_col=0)
movie

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,label_boxoffice
movie_id,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1,1
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1,1
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1,1
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1,6
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7881,We Need to Do Something,Unrated,97.0,48.0,4.5,Sean King O'Grady,Max Booth III,Vinessa Shaw,Horror,2021,1,0
7916,Uncharted,PG-13,116.0,42.0,6.9,Ruben Fleischer,Rafe Judkins,Tom Holland,Action,2022,1,0
7920,Moonfall,PG-13,130.0,41.0,5.3,Roland Emmerich,Spenser Cohen,Patrick Wilson,Action,2022,1,1


In [38]:
movie1 = movie.drop(['label_boxoffice'],axis=1)
movie1

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice
movie_id,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1
...,...,...,...,...,...,...,...,...,...,...,...
7881,We Need to Do Something,Unrated,97.0,48.0,4.5,Sean King O'Grady,Max Booth III,Vinessa Shaw,Horror,2021,1
7916,Uncharted,PG-13,116.0,42.0,6.9,Ruben Fleischer,Rafe Judkins,Tom Holland,Action,2022,1
7920,Moonfall,PG-13,130.0,41.0,5.3,Roland Emmerich,Spenser Cohen,Patrick Wilson,Action,2022,1


In [39]:
upcoming = pd.read_csv('predictions_upcoming_boxoffice.csv', index_col=0)
upcoming

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_Upcoming,label_boxoffice
movie_id,,,,,,,,,,,,
1,A Day to Die,0,0,0.0,0.0,Wes Miller,Scott Mallace,Kevin Dillon,Action,1970,1,0
2,He Was Asking for It: A Noir Film,0,0,0.0,0.0,Rocky Karlage,Staci Layne Wilson,Rocky Karlage,Action,2022,1,0
3,Tyson's Run,PG,103,0.0,0.0,Kim Bass,Kim Bass,Major Dodson,Action,2022,1,0
4,X,R,0,0.0,0.0,Ti West,Ti West,Mia Goth,Action,2022,1,0
5,Operation Fortune: Ruse de guerre,0,0,0.0,0.0,Guy Ritchie,Marn Davies,Josh Hartnett,Action,2022,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,1,0
79,Untitled Illumination Entertainment Project,0,0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,1,0
80,I Wanna Dance with Somebody,0,0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,1,0


In [40]:
#heroku_df= movie.join(upcoming, on='movie_id', how='left')
#heroku_df

heroku_df = pd.concat([movie1, upcoming], axis=0)
heroku_df

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,Predictions_Upcoming,label_boxoffice
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,NaN,NaN
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,NaN,NaN
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,NaN,NaN
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,NaN,NaN
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,NaN,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,NaN,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,NaN,1.0,0.0


In [41]:
heroku_df['Predictions_BoxOffice'] = heroku_df['Predictions_BoxOffice'].fillna(0)
heroku_df

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,Predictions_Upcoming,label_boxoffice
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,NaN,NaN
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,NaN,NaN
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,NaN,NaN
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,NaN,NaN
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,0.0,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,0.0,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,0.0,1.0,0.0


In [42]:
heroku_df['Predictions_Upcoming'] = heroku_df['Predictions_Upcoming'].fillna(0)
heroku_df

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,Predictions_Upcoming,label_boxoffice
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,0.0,NaN
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,0.0,NaN
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,0.0,NaN
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,0.0,NaN
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,0.0,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,0.0,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,0.0,1.0,0.0


In [43]:
heroku_df['label_boxoffice'] = heroku_df['label_boxoffice'].fillna(0)
heroku_df

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,Predictions_Upcoming,label_boxoffice
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,0.0,0.0
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,0.0,0.0
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,0.0,0.0
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,0.0,0.0
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,0.0,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,0.0,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,0.0,1.0,0.0


In [44]:
# save csv
heroku_df.to_csv("heroku_df.csv", index=False)
heroku_df

,title,movie_rating,run_time,meta_score,imdb_rating,director,writer,actor,genre_name,Year,Predictions_BoxOffice,Predictions_Upcoming,label_boxoffice
movie_id,,,,,,,,,,,,,
1,The Shining,R,146.0,0.0,8.4,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1980,1.0,0.0,0.0
9,Superman II,PG,127.0,0.0,6.8,Richard Lester,Jerry Siegel,Gene Hackman,Action,1981,1.0,0.0,0.0
15,Dressed to Kill,R,104.0,0.0,7.1,Brian De Palma,Brian De Palma,Michael Caine,Crime,1980,1.0,0.0,0.0
16,Somewhere in Time,PG,103.0,0.0,7.2,Jeannot Szwarc,Richard Matheson,Christopher Reeve,Drama,1980,1.0,0.0,0.0
18,9 to 5,PG,109.0,0.0,6.9,Colin Higgins,Patricia Resnick,Jane Fonda,Comedy,1980,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Aquaman and the Lost Kingdom,0,0.0,0.0,0.0,James Wan,Paul Norris,Temuera Morrison,Comedy,2022,0.0,1.0,0.0
79,Untitled Illumination Entertainment Project,0,0.0,0.0,0.0,Michael Jelenic,Shigeru Miyamoto,Seth Rogen,Drama,2022,0.0,1.0,0.0
80,I Wanna Dance with Somebody,0,0.0,0.0,0.0,Kasi Lemmons,Anthony McCarten,Tamara Tunie,Drama,2022,0.0,1.0,0.0


In [88]:
df = pd.read_csv('predictions_score.csv', index_col=0)
df

,rating,genre,year,released,votes,director,writer,star,country,gross,company,runtime,Predictions_imbd_score,label_imbd_score
name,,,,,,,,,,,,,,
Caddyshack,R,Comedy,1980,25-Jul-80,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,39846344,Orion Pictures,98.0,2,2
Friday the 13th,R,Horror,1980,9-May-80,123000.0,Sean S. Cunningham,Victor Miller,Betsy Palmer,United States,39754601,Paramount Pictures,95.0,1,1
Heaven's Gate,R,Adventure,1980,24-Apr-81,14000.0,Michael Cimino,Michael Cimino,Kris Kristofferson,United States,3484523,Partisan Productions,219.0,1,1
The Final Countdown,PG,Action,1980,1-Aug-80,22000.0,Don Taylor,Thomas Hunter,Kirk Douglas,United States,16647800,Bryna Productions,103.0,1,1
American Gigolo,R,Crime,1980,1-Feb-80,22000.0,Paul Schrader,Paul Schrader,Richard Gere,United States,22743674,Paramount Pictures,117.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Ground Beneath My Feet,Not Rated,Drama,2019,26-Jul-19,898.0,Marie Kreutzer,Marie Kreutzer,Valerie Pachner,Austria,29251,Novotny & Novotny Filmproduktion GmbH,108.0,1,1
India's Most Wanted,Not Rated,Action,2019,24-May-19,2000.0,Raj Kumar Gupta,Raj Kumar Gupta,Arjun Kapoor,India,1715851,Fox STAR Studios,123.0,1,0
Tenet,PG-13,Action,2020,3-Sep-20,387000.0,Christopher Nolan,Christopher Nolan,John David Washington,United States,363656624,Warner Bros.,150.0,2,2


In [92]:
df['movie_id'] = range(1, len(df) + 1)
df

,rating,genre,year,released,votes,director,writer,star,country,gross,company,runtime,Predictions_imbd_score,label_imbd_score,movie_id
name,,,,,,,,,,,,,,,
Caddyshack,R,Comedy,1980,25-Jul-80,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,39846344,Orion Pictures,98.0,2,2,1
Friday the 13th,R,Horror,1980,9-May-80,123000.0,Sean S. Cunningham,Victor Miller,Betsy Palmer,United States,39754601,Paramount Pictures,95.0,1,1,2
Heaven's Gate,R,Adventure,1980,24-Apr-81,14000.0,Michael Cimino,Michael Cimino,Kris Kristofferson,United States,3484523,Partisan Productions,219.0,1,1,3
The Final Countdown,PG,Action,1980,1-Aug-80,22000.0,Don Taylor,Thomas Hunter,Kirk Douglas,United States,16647800,Bryna Productions,103.0,1,1,4
American Gigolo,R,Crime,1980,1-Feb-80,22000.0,Paul Schrader,Paul Schrader,Richard Gere,United States,22743674,Paramount Pictures,117.0,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Ground Beneath My Feet,Not Rated,Drama,2019,26-Jul-19,898.0,Marie Kreutzer,Marie Kreutzer,Valerie Pachner,Austria,29251,Novotny & Novotny Filmproduktion GmbH,108.0,1,1,1849
India's Most Wanted,Not Rated,Action,2019,24-May-19,2000.0,Raj Kumar Gupta,Raj Kumar Gupta,Arjun Kapoor,India,1715851,Fox STAR Studios,123.0,1,0,1850
Tenet,PG-13,Action,2020,3-Sep-20,387000.0,Christopher Nolan,Christopher Nolan,John David Washington,United States,363656624,Warner Bros.,150.0,2,2,1851


In [97]:
df.set_index(df["movie_id"], drop = False, inplace = True)

In [94]:
movie = pd.DataFrame(df[['movie_id','name','rating','genre','released','votes','director','writer','star','company',
                        'runtime','Predictions_imbd_score','label_imbd_score']])
movie

KeyError: "['name'] not in index"

In [50]:
rating_movie1.set_index('movie_id')

,rating,genre,year,released,votes,director,writer,star,country,gross,company,runtime,Predictions_imbd_score,label_imbd_score
movie_id,,,,,,,,,,,,,,
0,R,Comedy,1980,25-Jul-80,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,39846344,Orion Pictures,98.0,2,2
1,R,Horror,1980,9-May-80,123000.0,Sean S. Cunningham,Victor Miller,Betsy Palmer,United States,39754601,Paramount Pictures,95.0,1,1
2,R,Adventure,1980,24-Apr-81,14000.0,Michael Cimino,Michael Cimino,Kris Kristofferson,United States,3484523,Partisan Productions,219.0,1,1
3,PG,Action,1980,1-Aug-80,22000.0,Don Taylor,Thomas Hunter,Kirk Douglas,United States,16647800,Bryna Productions,103.0,1,1
4,R,Crime,1980,1-Feb-80,22000.0,Paul Schrader,Paul Schrader,Richard Gere,United States,22743674,Paramount Pictures,117.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,Not Rated,Drama,2019,26-Jul-19,898.0,Marie Kreutzer,Marie Kreutzer,Valerie Pachner,Austria,29251,Novotny & Novotny Filmproduktion GmbH,108.0,1,1
1849,Not Rated,Action,2019,24-May-19,2000.0,Raj Kumar Gupta,Raj Kumar Gupta,Arjun Kapoor,India,1715851,Fox STAR Studios,123.0,1,0
1850,PG-13,Action,2020,3-Sep-20,387000.0,Christopher Nolan,Christopher Nolan,John David Washington,United States,363656624,Warner Bros.,150.0,2,2


In [48]:
rating_movie2 = rating_movie1.set_index('movie_id')
rating_movie2

AttributeError: 'numpy.ndarray' object has no attribute 'set_index'

In [49]:
rating_movie1 = rating_movie.rename(columns={"name": "title"})
rating_movie1

,rating,genre,year,released,votes,director,writer,star,country,gross,company,runtime,Predictions_imbd_score,label_imbd_score,movie_id
name,,,,,,,,,,,,,,,
Caddyshack,R,Comedy,1980,25-Jul-80,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,39846344,Orion Pictures,98.0,2,2,0
Friday the 13th,R,Horror,1980,9-May-80,123000.0,Sean S. Cunningham,Victor Miller,Betsy Palmer,United States,39754601,Paramount Pictures,95.0,1,1,1
Heaven's Gate,R,Adventure,1980,24-Apr-81,14000.0,Michael Cimino,Michael Cimino,Kris Kristofferson,United States,3484523,Partisan Productions,219.0,1,1,2
The Final Countdown,PG,Action,1980,1-Aug-80,22000.0,Don Taylor,Thomas Hunter,Kirk Douglas,United States,16647800,Bryna Productions,103.0,1,1,3
American Gigolo,R,Crime,1980,1-Feb-80,22000.0,Paul Schrader,Paul Schrader,Richard Gere,United States,22743674,Paramount Pictures,117.0,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Ground Beneath My Feet,Not Rated,Drama,2019,26-Jul-19,898.0,Marie Kreutzer,Marie Kreutzer,Valerie Pachner,Austria,29251,Novotny & Novotny Filmproduktion GmbH,108.0,1,1,1848
India's Most Wanted,Not Rated,Action,2019,24-May-19,2000.0,Raj Kumar Gupta,Raj Kumar Gupta,Arjun Kapoor,India,1715851,Fox STAR Studios,123.0,1,0,1849
Tenet,PG-13,Action,2020,3-Sep-20,387000.0,Christopher Nolan,Christopher Nolan,John David Washington,United States,363656624,Warner Bros.,150.0,2,2,1850
